In [1]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
spark.conf.set("sprk.sql.parquet.vorder.enabled","true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.enabled", "true")
spark.conf.set("spark.microsoft.delta.optimizeWrite.binSize", "1073741824")

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 3, Finished, Available)

In [2]:
df = spark.read.format("parquet").load('Files/TCPH-RAW/Customers/*')
display(df.limit(10))

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 4, Finished, Available)

SynapseWidget(Synapse.DataFrame, d54b43d6-8dfb-46d7-ab56-fb8ede339d17)

In [4]:
df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/orders.tbl_00000.1.parquet')
display(df.limit(100))

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, f69ec5cc-77b6-4e90-aa2f-e11e9d1624f5)

In [5]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_order'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
df = df.withColumn('Year', year(col("o_orderdate")))
df = df.withColumn('Quarter', quarter(col("o_orderdate")))
df = df.withColumn('Month', month(col("o_orderdate")))

df.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)


StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 7, Finished, Available)

In [8]:
df = spark.sql("SELECT Year, Quarter, Month, COUNT(o_orderkey) FROM FabricLakehouse.fact_order group by Year, Quarter, Month order by Year, Quarter, Month")
display(df)

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 10, Finished, Available)

SynapseWidget(Synapse.DataFrame, 5faca30a-4c42-488b-a7f4-4977ba6abe25)

In [9]:
table_name = 'dim_customer'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Customers/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 11, Finished, Available)

In [10]:
table_name = 'dim_nation'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Nation/*')

df.write.mode("overwrite").format("delta").save("Tables/" + table_name)

StatementMeta(, 0cf73c52-ea2b-472d-b72c-3378a49f5f5d, 12, Finished, Available)

In [3]:
table_name = 'dim_region_c'

df = spark.read.format("parquet").load('Files/TCPH-RAW/Region/*')

df.write.mode("overwrite").option("overwriteSchema","true").format("delta").save("Tables/" + table_name)

StatementMeta(, 1c260394-ba02-411f-bcbf-b473f71f341c, 5, Finished, Available)

In [2]:
from pyspark.sql.functions import col, year, month, quarter

table_name = 'fact_orderline'

dforder = spark.read.format("parquet").load('Files/TCPH-RAW/Orders/*')
dforder = dforder.withColumn('Year', year(col("o_orderdate")))
dforder = dforder.withColumn('Quarter', quarter(col("o_orderdate")))
dforder = dforder.withColumn('Month', month(col("o_orderdate")))

dfline = spark.read.format("parquet").load('Files/TCPH-RAW/LineItems/*')

dforderline = dforder.join(dfline,dforder.o_orderkey == dfline.l_orderkey, how='Inner')

dforderline.write.mode("overwrite").format("delta").partitionBy("Year","Quarter").save("Tables/" + table_name)

StatementMeta(, 82d8e7ee-fde6-4bd9-912c-85583fdedc74, 4, Finished, Available)

In [3]:
dforderline.count()

StatementMeta(, 82d8e7ee-fde6-4bd9-912c-85583fdedc74, 5, Finished, Available)

3000028242